<a href="https://colab.research.google.com/github/NumanAloko/ML-for-CFS-Built-up-Columns/blob/main/MLP_Classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Training and testing of Multilayer Perceptron (MLP) 0f Artificial Neural Networks classification model**

In [ ]:
# Import necessary libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV,learning_curve

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [ ]:
############################### Create Dataframe ###############################

# Load the dataset
file_path = r'C:\Users\rq22807\OneDrive - University of Bristol\Desktop\ML Paper for Revision\GitHub\CFS_Built-up_Columns_ML_Dataset.xlsx'
df = pd.read_excel(file_path, sheet_name='L and LG Interact')

# Remove  whitespaces between letters in names of buckling failure modes
df['FM'] = df['FM'].str.strip()

# label encoding
le = LabelEncoder()
# Encode the 'Failure modes' name to numbers
df['FM'] = le.fit_transform(df['FM'])
data_x= df[['L','t','h','b','KL_r','A','Py','Pne','P(crl_s,crd_s)']] # parameters for Partial composite Approach

# Create the feature DataFrame
X= pd.DataFrame(df,columns=data_x.columns) #input variables

# Data Scaling
sc_X = StandardScaler()
X_scaled = sc_X.fit_transform(X)

# Split the data into train and test sets for the classification task
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_scaled, df['FM'], test_size=0.3, random_state=123)

In [ ]:
# Define the Multilayer Perceptron (MLP) 0f Artificial Neural Networks classification model

# Hyperparameter tuning based on Grid search method
# Define the parameter grid
#param_grid = {
    #'hidden_layer_sizes': [(20, 30, 20, 20, 30), (100, 50), (100,), (50, 100, 50), (150, 100, 50)],
    #'activation': ['relu', 'tanh', 'logistic'],
    #'solver': ['adam', 'sgd'],
    #'alpha': [0.0001,0.001,0.005, 0.01],
   #'learning_rate': ['constant', 'adaptive'],
    #'learning_rate_init': [0.001, 0.01, 0.1]
#}
#mlp = MLPClassifier(max_iter=2000, tol=0.0001, random_state=123, early_stopping=True, validation_fraction=0.3, n_iter_no_change=20)

# Optimised Hyperparameters (Best parameters selected after tuning)
param_grid = {
    'hidden_layer_sizes': [(50, 100, 50)],
    'activation': ['tanh'],
    'solver': ['adam'],
    'alpha': [0.0001],
    'learning_rate': ['constant'],
    #'learning_rate_init': [0.1]
}

mlp = MLPClassifier(max_iter=2000, tol=0.0001, random_state=123)
clf = GridSearchCV(estimator=mlp, cv=10, param_grid=param_grid, n_jobs=-1)

# Train the classifier on feature and target data
clf.fit(X_train_class, y_train_class)

# View the best parameters for the model found using grid search
print('Best hidden_layer_sizes:',clf.best_estimator_.hidden_layer_sizes)
print('Best activation:',clf.best_estimator_.activation)
print('Best solver:',clf.best_estimator_.solver)
print('Best alpha:',clf.best_estimator_.alpha)
print('Best learning_rate:',clf.best_estimator_.learning_rate)

# Make predictions
y_pred_class_train = clf.predict(X_train_class)
y_pred_class_test = clf.predict(X_test_class)

In [ ]:
############################################ Performance Metrics ##########################
# Calculate accuracy
accuracy_train = accuracy_score(y_train_class, y_pred_class_train)
accuracy_test = accuracy_score(y_test_class, y_pred_class_test)

print('Training accuracy: {:.2f}'.format(accuracy_train))
print('Test accuracy: {:.2f}'.format(accuracy_test))

# Print confusion matrix
print('Confusion matrix:')
print(confusion_matrix(y_test_class, y_pred_class_test))

# Print classification report
print('Classification report:')
print(classification_report(y_test_class, y_pred_class_test))

#  class names (Inverse Encoding)
class_names = le.inverse_transform(np.unique(y_test_class))
# Print the class names
print('Classes:', class_names)
# Get the mapping of class names to numbers
class_mapping = {label: index for index, label in enumerate(le.classes_)}
# Print the mapping
print('Encoding:', class_mapping)

**Classify the failure mode of a new instance after training of MLP ML Model**

In [ ]:
# Make new prediction
# Manually input the new data
new_data = {
    'L': [1500],         # Length(mm)
    't': [1.5],          # Thickness of the section (mm)
    'h': [179],          # Height of the section (mm)
    'b': [67],           # Flange of the section (mm)
    'KL_r': [29.39],     # Non-dimensioanl member slenderness
    'Py': [418.5],       # Yielding Strength of Section (kN)
    'A': [930],          # Area (mm2) of the section
    'Pne': [393.62],     # Global Buckling Strength of the Section (kN)
    'P(crl_s,crd_s)': [47.74]  # Sectional (Local or Distortional) Buckling Strength of the section (kN)
}

# Convert the new input data to a DataFrame
new_input_data = pd.DataFrame(new_data)

# Ensure the new input data has the same columns as the training data
data_x_columns = ['L','t','h','b','KL_r','A','Py','Pne','P(crl_s,crd_s)']
new_input_data = new_input_data[data_x_columns]

# Scale the new input data using the same scaler used for training data
sc_X = StandardScaler()
sc_X.fit(X)  # Fit the scaler on the original training data
new_input_scaled = sc_X.transform(new_input_data)

# Make predictions
predicted_class = clf.predict(new_input_scaled)
#predicted_probabilities = clf.predict_proba(new_input_scaled)

# Decode the predicted class
predicted_class_decoded = le.inverse_transform(predicted_class)

# Print the results
print('Predicted class:', predicted_class_decoded[0])

**Interpretation of the new instance**

In [ ]:
# MLP Classification Model Explainer
import lime
import lime.lime_tabular

# Create a LIME explainer
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train_class,
    feature_names=X.columns,
    class_names=class_names,
    mode='classification'
)

# New input data
exp = explainer.explain_instance(
    data_row=new_input_scaled[0],
    predict_fn=clf.predict_proba
)

# Show the explanation
exp.show_in_notebook(show_table=True, show_all=False)

# Save the explanation as an HTML file
exp.save_to_file('lime_explanation_new_data1.html')